## LICENSE: GPL 3.0
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.\
Voxel-based Neural Style Transfer\
Copyright (c)

Honda Research Institute Europe GmbH\
Authors: Timo Friedrich\
Contact: timo.friedrich@honda-ri.de

# Voxel-based Neural Style Transfer
We provide the code to reproduce our voxel-based Neural Style Transfer results from the following publications.
- Friedrich, T., Wollstadt, P., & Menzel, S. (2020). The Effects of Non-linear Operators in Voxel-Based Deep Neural Networks for 3D Style Reconstruction. 2020 IEEE Symposium Series on Computational Intelligence (SSCI), 1460–1468. https://doi.org/10.1109/SSCI47803.2020.9308308

- Friedrich, T., Hammer, B., & Menzel, S. (2021). Voxel-Based Three-Dimensional Neural Style Transfer. 16th International Work-Conference on Artificial Neural Networks, IWANN 2021, 334–346. https://doi.org/10.1007/978-3-030-85030-2_28

The classification network Net_belu from our papers is included as well as some programmatically created voxel models. We also provide a voxelized Stanford bunny with a resolution of 256 voxel per dimension. 
http://graphics.stanford.edu/data/3Dscanrep/

---
## Environment
Create a compatible Conda Environment:

```
conda create --name EigenEnv --file requirements.txt
conda activate EigenEnv
```

You need a machine with sufficient GPU memory. We used a nVidia Quadro RTX8000 with >40GB memory.

---
## Additional Data
In case you like to try other 3D objects, we refer to the [ModelNet40](https://modelnet.cs.princeton.edu/) dataset.
Please use the following commands to voxelize the mesh-based models.
[Binvox voxelization tool](https://www.patrickmin.com/binvox/)

Command: 
Either use:
```
binvox -aw -dc -cb -pb -down -d 448 modelnet40object.off
```
and manually add a 16 voxel padding programmatically before applying style transfer (recommended) or use:
```
binvox -aw -dc -cb -pb -down -d 512 modelnet40object.off
```
but consider that the actual voxel objects extends now to the maximum voxel expansion which has negativ effects during Neural Style Transfer.

---

## Setup


### Import and configure modules

In [ ]:
import IPython.display as display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime
from time import sleep
import functools
import shutil
import pickle
from sklearn.neighbors import KDTree, BallTree
import os
os.putenv('AUTOGRAPH_VERBOSITY', '10')
#from geometric_learning.voxel.helpers import *
import pymrt as pm # version 0.0.2.5!
import pymrt.geometry
import tensorflow as tf
from scipy import ndimage
import yaml
import binvox_rw


# Data and Voxel models
The following function provides compatible voxel models by a name.

In [ ]:
def get_voxel(name):
    ''' 
    Returns voxel model in TensorFlow input layer format
    
    name (str): name of the to be received voxel model 
    '''
    r=256
    
    if name == 'sphere81':
        v = pm.geometry.sphere((r,r,r), 81 , 0.5).astype(np.float32) # sphere, similar volume as bunny
        return v[np.newaxis,:,:,:,np.newaxis] 
    
    elif name == 'cube131':
        v = pm.geometry.cube((r,r,r), 131 , 0.5).astype(np.float32) #cube, , similar volume as bunny
        return v[np.newaxis,:,:,:,np.newaxis] 
    
    elif name == 'cubic6028':
        # target mass = bunny
        cb = np.zeros([256,256,256], dtype=np.float32)
        n = 60 # half edge ength
        cb[128-n:128+n, 128-n:128+n, 128-n:128+n] = 1
        m = 28
        cb[128+n:128+n+m, 128-m:128+m,    128-m:128+m] = 1
        cb[128-m:128+m,   128+n:128+n+m,  128-m:128+m] = 1
        cb[128-m:128+m,   128-m:128+m,    128+n:128+n+m] = 1
        cb[128-n-m:128-n, 128-m:128+m,   128-m:128+m] = 1
        cb[128-m:128+m,   128-n-m:128-n, 128-m:128+m] = 1
        cb[128-m:128+m,   128-m:128+m,   128-n-m:128-n] = 1
        return cb[np.newaxis,:,:,:,np.newaxis]    
  
    elif name == 'bunny224':
        with open('./data/stanford_bunny_224.binvox', 'rb') as f:
            vj = binvox_rw.read_as_3d_array(f)
            vj = np.pad(vj.data, pad_width=16, mode='constant', constant_values=0)
        return vj[np.newaxis,:,:,:,np.newaxis].astype(np.float32)
           
    else:
        raise ValueError(f'{name} not known yet')

## ST Code Functions

In [ ]:
# A Keras model that returns the style and content tensors for the given layer of the provided neural network.
class ResponseExtractor(tf.keras.models.Model):
    def __init__(self, vnet_file, style_layer_names, content_layer_names ):
        """
        vnet_file             (str):       Path of a saved Keras classification network model
        style_layer_names     (list(str)): Layer names for style map extraction
        content_layer_names:  (list(str)): Layer names for content map extraction
        """
        super(ResponseExtractor, self).__init__()
        
        self.vnet = tf.keras.models.load_model(vnet_file)
        self.vnet.trainable = False
        
        layer_names = style_layer_names + content_layer_names
        output_layers = [self.vnet.get_layer(name).output for name in layer_names]
        self.vnetLayerResponse =  tf.keras.Model([self.vnet.input], output_layers)
        self.vnetLayerResponse.trainable = False
        
        self.style_layer_names = style_layer_names
        self.content_layer_names = content_layer_names
        self.num_style_layers = len(style_layer_names)


    def call(self, voxel):
        "Expects float input in [0,1]"
        
        layer_responses = self.vnetLayerResponse(voxel)
        style_layer_responses, content_layer_responses = (layer_responses[:self.num_style_layers], 
                                                        layer_responses[self.num_style_layers:])
        return style_layer_responses, content_layer_responses
    

In [ ]:
# Keras Model which calculates the Style Transfer loss value including the regularization terms.
# The init function does precomputation if necessary to speed up the optimization loop
class LossHandler(tf.keras.models.Model):
    def __init__(self, 
                 map_extractor,
                 style_voxel,
                 content_voxel,
                 content_loss_type,
                 style_loss_type, 
                 num_content_layers, 
                 num_style_layers,
                 style_weight,
                 style_layer_weights,
                 content_weight,
                 total_variation_weight,
                 dist_reg_weight,
                 dist_reg_padding,
                 sym_reg_weight,
                 bor_reg_weight,
                 bor_reg_margin,
                ):
        super(LossHandler, self).__init__()   

        self.response_extractor = response_extractor
        self.style_voxel = style_voxel
        self.content_voxel = content_voxel
        self.content_loss_type = content_loss_type 
        self.style_loss_type = style_loss_type
        self.num_content_layers = num_content_layers
        self.num_style_layers = num_style_layers
        self.style_weight = style_weight
        self.style_layer_weights = style_layer_weights
        self.content_weight = content_weight
        self.total_variation_weight = total_variation_weight
        self.dist_reg_weight = dist_reg_weight
        self.dist_reg_padding = dist_reg_padding
        self.sym_reg_weight = sym_reg_weight 
        self.bor_reg_weight = bor_reg_weight
        self.bor_reg_margin = bor_reg_margin
        
        self.dist_matrix = None
        self.borreg_matrix = None  
        
        # setup loss functions
        if self.content_loss_type == 'gatys':
            self.content_loss_fcn = self.content_loss_gatys
        else:
            raise NotImplemented
            
        if self.style_loss_type == 'hri_std':
            self.style_loss_fcn = self.style_loss_gatys
            self.style_gram_fcn = self.gram_matrix_hri_standardized
        elif self.style_loss_type == 'gatys':
            self.style_loss_fcn = self.style_loss_gatys
            self.style_gram_fcn = self.gram_matrix_gatys
        else:
            raise NotImplemented
            
        ### Precalculations    
        
        # get the constant style and content values of the targets
        self.style_responses, _  = self.response_extractor(self.style_voxel)
        _, self.content_responses  = self.response_extractor(self.content_voxel)
        # calc constant grams for the style target shape    
        self.style_grams = [self.style_gram_fcn(r) for r in self.style_responses]
                
        # distant material to content regularization
        if self.dist_reg_weight > 0:
            print('Caluclating distance matrix for the given content shape. This can take some minutes')
            # precalc the distance matrix
            dist_matrix = get_distance_matrix(self.content_voxel.numpy()[0,:,:,:,0].squeeze(), padding=self.dist_reg_padding)
    #         dist_matrix = dist_matrix / np.max(dist_matrix)
            self.dist_matrix = tf.Variable(dist_matrix[np.newaxis,:,:,:,np.newaxis].astype(np.float32))
    
        # border regulariztion matrix
        if self.bor_reg_weight > 0:
            self.borreg_matrix = get_borreg_matrix(self.content_voxel.numpy().shape, self.bor_reg_margin)
    
    def call(self, itshape_voxel):
        """call to get losses for the current iterated shape"""
        itshape_style_responses, itshape_content_responses = self.response_extractor(itshape_voxel)
        
        # TODO choose which variant here
        content_loss = self.content_loss_fcn(itshape_content_responses)
        style_loss = self.style_loss_fcn(itshape_style_responses)
        
        # Regularization terms
        loss_totalvar = 0
        if self.total_variation_weight > 0:
            loss_totalvar = self.total_variation_weight * total_variation_loss(voxel)          

        loss_distreg = 0
        if self.dist_reg_weight > 0:
            loss_distreg = tf.math.abs(itshape_voxel - self.content_voxel)
            loss_distreg = tf.math.multiply(loss_distreg, self.dist_matrix) 
            loss_distreg = tf.math.reduce_sum(loss_distreg) * self.dist_reg_weight

        loss_symreg = 0
        if self.sym_reg_weight > 0:
            loss_symreg = self.sym_reg_weight * symmetry_loss(itshape_voxel)

        loss_borreg = 0
        if self.bor_reg_weight > 0:
            loss_borreg = tf.math.reduce_sum(tf.math.multiply(self.borreg_matrix, itshape_voxel))
            loss_borreg = self.bor_reg_weight * loss_borreg
            
        total =  style_loss + content_loss + loss_totalvar + loss_distreg + loss_symreg + loss_borreg
        
        return {'total': total,
                'style': style_loss,
                'content': content_loss,
                'reg_tvar': loss_totalvar,
                'reg_dist': loss_distreg,
                'reg_sym': loss_symreg,
                'reg_bor': loss_borreg,
               }
    
    # gram matrix and style loss function applied in the LossHandler
    def gram_matrix(self, input_tensor):
        """raw gram matrix calculation"""
        return tf.linalg.einsum('bijkc,bijkd->bcd', input_tensor, input_tensor)
    
    def gram_matrix_gatys(self, input_tensor):
        """original gram calc by gatys, k added for 3rd dimension"""
        result = self.gram_matrix(input_tensor)
        input_shape = tf.shape(input_tensor)
        num_locations = tf.cast(input_shape[1]*input_shape[2]*input_shape[3], tf.float32)
        return result/(num_locations)

    def gram_matrix_hri_standardized(self, input_tensor):
        """hri standardized gram calc"""
        result = self.gram_matrix(input_tensor)
        result = (result - tf.math.reduce_mean(result)) / tf.math.reduce_std(result)
        return result * 1000

    def style_loss_gatys(self, responses): 
        """style loss gatys"""
        grams = [self.style_gram_fcn(r) for r in responses]
        style_loss = tf.add_n([slw * tf.reduce_mean((g1-g2)**2) for g1, g2, slw in zip(grams, self.style_grams, self.style_layer_weights)])
        style_loss *= self.style_weight / self.num_style_layers
        return style_loss

    def content_loss_gatys(self, response):
        """content loss gatys"""
        content_loss = tf.add_n([tf.reduce_mean((r1-r2)**2) for r1, r2 in zip(response, self.content_responses)])
        content_loss *= self.content_weight / self.num_content_layers
        return content_loss      

In [ ]:
# regularization related functions applied in the LossHandler  
def clip_0_1(voxel):
    return tf.clip_by_value(voxel, clip_value_min=0.0, clip_value_max=1.0)


def high_pass_x_y_z(voxel):
    z_var = voxel[:,:,:,1:,:] - voxel[:,:,:,:-1,:]
    y_var = voxel[:,:,1:,:,:] - voxel[:,:,:-1,:,:]
    x_var = voxel[:,1:,:,:,:] - voxel[:,:-1,:,:,:]
    return x_var, y_var, z_var


def total_variation_loss(voxel):
    x_deltas, y_deltas, z_deltas = high_pass_x_y_z(voxel)
    return tf.reduce_mean(x_deltas**2) + tf.reduce_mean(y_deltas**2) + tf.reduce_mean(z_deltas**2)


def symmetry_loss(v):
    """Used for symmetry enforcement of ModelNEt 40 cars, sym along y axis"""
    vsize = 256 #v.shape[2]
    vh1 = tf.slice(v, [0,0,0,0,0], [1,vsize,vsize//2,vsize,1]) # split in y axis, left/right ModelNet40 cars
    vh2 = tf.slice(v, [0,0,vsize//2,0,0], [1,vsize,vsize//2,vsize,1])
    vh2 = tf.reverse(vh2, axis=[2])
    loss_symreg = tf.reduce_mean(tf.math.square(vh1 - vh2))
    return loss_symreg


def get_distance_matrix(v, tree_type='kd', padding=16):
    # returns a 3D matrix containing distance values to the voxel-shape-surface with a neutral padding zone around the shape
    
    # Create point set for surface voxels and all zero voxels outside the model
    v_surf = np.copy(v)
    v_temp = ndimage.minimum_filter(v, size=3)
    v_surf = v - v_temp
    psurf = np.array(np.nonzero(v_surf)).transpose()
    v_void = -(v_surf-1)
    pvoid = np.array(np.nonzero(v_void)).transpose()

    #sklearn trees
    if tree_type == 'kd':
        tree = KDTree(psurf)
    elif tree_type == 'ball':
        tree = BallTree(psurf)
    else:
        raise ValueError('Unknown value for tree_type') 
    
    pvoid_dist, _ = tree.query(pvoid, k=1)
    pvoid_dist = pvoid_dist.squeeze()
    
    # pcl -> matrix
    pvoid_dist_matrix = np.zeros(v.shape)
    pvoid_dist_matrix[tuple(pvoid.T)] = pvoid_dist
    
    pvoid_dist_matrix -= padding
    pvoid_dist_matrix[pvoid_dist_matrix < 0] = 0   
    return pvoid_dist_matrix


def get_borreg_matrix(shape, margin):
    # return a 3D matrix with penalty values on the border region
    b = margin
    borreg_matrix = np.zeros(shape).astype(np.float32)
    borreg_matrix[0,0:b,:,:,0] = 1
    borreg_matrix[0,:,0:b,:,0] = 1
    borreg_matrix[0,:,:,0:b,0] = 1
    borreg_matrix[0,-b:,:,:,0] = 1
    borreg_matrix[0,:,-b:,:,0] = 1
    borreg_matrix[0,:,:,-b:,0] = 1
    return tf.constant(borreg_matrix)

# Experiment LOOP Config

In [ ]:
EXPERIMENT_NAME = 'myExperiment'

In [ ]:
# each value has to be wrapped in a list ! even if single value
loop_conf = dict()
loop_conf['seed'] = [13]
loop_conf['save_modulo'] = [9999999999]

loop_conf['style_loss_type'] = ['hri_std'] #hri_std, gatys
loop_conf['content_loss_type'] = ['gatys']

loop_conf['content_voxel'] = ['bunny224'] 
loop_conf['style_voxel'] = [ 'cube131'] 
loop_conf['initial_voxel'] = ['bunny224'] 

loop_conf['content_layer_names'] = [
#     ['tf_op_layer_concat'],
#     ['tf_op_layer_concat_1'],
#     ['tf_op_layer_concat_2'],
#     ['tf_op_layer_concat_3'],
    ['tf_op_layer_concat_4'],
#     ['tf_op_layer_concat_5'],
#     ['tf_op_layer_concat_2'],
#     ['tf_op_layer_concat_1'],
#     ['tf_op_layer_concat_1'],
]

loop_conf['style_layer_names'] = [
#     ['tf_op_layer_concat'],
#     ['tf_op_layer_concat', 'tf_op_layer_concat_1'],
#     ['tf_op_layer_concat', 'tf_op_layer_concat_1', 'tf_op_layer_concat_2'],
#     ['tf_op_layer_concat', 'tf_op_layer_concat_1', 'tf_op_layer_concat_2', 'tf_op_layer_concat_3'],
    ['tf_op_layer_concat', 'tf_op_layer_concat_1', 'tf_op_layer_concat_2', 'tf_op_layer_concat_3', 'tf_op_layer_concat_4'],
#     ['tf_op_layer_concat', 'tf_op_layer_concat_1', 'tf_op_layer_concat_2', 'tf_op_layer_concat_3', 'tf_op_layer_concat_4', 'tf_op_layer_concat_5'],
#     ['tf_op_layer_concat_1', 'tf_op_layer_concat_2', 'tf_op_layer_concat_3', 'tf_op_layer_concat_4'],
]
loop_conf['style_layer_weights'] = [ # keep it sum = num(layers)
    [1,1,1,1,1,1,1,1],
#     [2,1,0.5,0.5],
#     [0.5,0.5,1,2],
]

loop_conf['optimizer'] = ['adam']
loop_conf['lr'] = [0.01]
loop_conf['epochs'] = [2]#[40]
loop_conf['steps_per_epoch'] = [2]#[500]
loop_conf['style_weight'] = [20]
loop_conf['content_weight'] = [4]
loop_conf['total_variation_weight'] = [0] 
loop_conf['dist_reg_weight'] = [0]#[500] 
loop_conf['dist_reg_padding'] = [15]
loop_conf['sym_reg_weight'] = [0] #[1e7]
loop_conf['bor_reg_weight'] = [10]
loop_conf['bor_reg_margin'] = [2]

In [ ]:
# loop_conf sanity check
for lk, lv in loop_conf.items():
    assert type(lv) == list, f"{lk} entry not a list"
l = [len(l) for l in loop_conf.values()]
l = set(l) 
ltest = l - set([1])
assert len(ltest) <= 1 , f"Misconfiguration of loop_conf. Multiple non 1 values found: {l}"

# transform into list of dicts for the loop
lc = []
for i in range(max(l)):
    lc.append(dict())
    for k in loop_conf.keys():
        lc[i][k] = loop_conf[k][min(i, len(loop_conf[k])-1 )]

# Global Configuration

In [ ]:
conf_global = dict()


# Documentation folder
# Folder where the voxel models are stored as pickle files.
# A copy of the executed NB is saved as an html file.
conf_global['doc_dir_base'] = './results'


# conf_global['vnet_file'] = f'{LOG_DIR}/model.38-0.48.hdf5'
conf_global['vnet_file'] = f'./data/Net_belu.hdf5'
conf_global['now'] = datetime.datetime.now()
conf_global['experiment'] = EXPERIMENT_NAME
conf_global['doc_dir'] = f"{conf_global['doc_dir_base']}/{conf_global['now']:%Y%m%d_%H%M%S}_{conf_global['experiment']}"

# Style Transfer Optimization Loop

In [ ]:
%%time

print("Output directory:")
print(conf_global['doc_dir'])
print()

# create folder for documentation
os.makedirs(conf_global['doc_dir'], exist_ok=True)
assert(os.path.exists(conf_global['doc_dir']))


#############################################################
# The configurations loop
for i,c in enumerate(lc):
    print('#'*10)
    print(f'Start Loop: {i}')
    
    # clear TF
    tf.keras.backend.clear_session()

    # set seeds
    tf.random.set_seed(c['seed'])
    np.random.seed(c['seed'])

    # Optimizer
    if c['optimizer'] == 'adam':
        opt = tf.optimizers.Adam(learning_rate=c['lr'], beta_1=0.99, epsilon=1e-1)
    elif c['optimizer'] == 'sgd':
        opt = tf.optimizers.SGD(learning_rate=c['lr'])
    else:
        raise ValueError('Unknown value for optimizer')

    # Voxel shapes
    i_voxel = tf.Variable(get_voxel(c['initial_voxel']))
    s_voxel = tf.constant(get_voxel(c['style_voxel']))
    c_voxel = tf.constant(get_voxel(c['content_voxel']))  
    
    # storing voxel shapes
    pickle.dump( {"voxel": np.squeeze(i_voxel.numpy())} , open( os.path.join(conf_global['doc_dir'], f"initial_{i}.pkl" ), "wb" ) )
    pickle.dump( {"voxel": np.squeeze(c_voxel.numpy())} , open( os.path.join(conf_global['doc_dir'], f"content_{i}.pkl" ), "wb" ) )
    pickle.dump( {"voxel": np.squeeze(s_voxel.numpy())} , open( os.path.join(conf_global['doc_dir'], f"style_{i}.pkl" ), "wb" ) )

    # get activation map extractor (includes neural networks)
    response_extractor = ResponseExtractor(conf_global['vnet_file'], c['style_layer_names'], c['content_layer_names'])
    
    # loss functions
    loss_handler = LossHandler(response_extractor,
                               s_voxel,
                               c_voxel,
                               content_loss_type = c['content_loss_type'],
                               style_loss_type = c['style_loss_type'],
                               num_content_layers = len(c['content_layer_names']),
                               num_style_layers = len(c['style_layer_names']),
                               style_weight = c['style_weight'],
                               style_layer_weights=c['style_layer_weights'],
                               content_weight = c['content_weight'],
                               total_variation_weight = c['total_variation_weight'],
                               dist_reg_weight = c['dist_reg_weight'],
                               dist_reg_padding = c['dist_reg_padding'],
                               sym_reg_weight = c['sym_reg_weight'],
                               bor_reg_weight = c['bor_reg_weight'],
                               bor_reg_margin = c['bor_reg_margin'],
                              )
    
    # training function
    @tf.function()
    def train_step(voxel, debug=False):
        with tf.GradientTape() as tape:
            losses = loss_handler(voxel)
        grad = tape.gradient(losses['total'], voxel)
        opt.apply_gradients([(grad, voxel)])
        voxel.assign(clip_0_1(voxel))
        return losses


    # the optimization
    voxel = i_voxel
    losses = train_step(voxel, True)
    
    # loss history and printing
    losshist = dict()
    for k in losses.keys():
        losshist[k] = [losses[k].numpy()]
    print("Initial losses:  ", end="")
    for k,v in losshist.items():
        print(f"{k}: {v[-1]:10.1f}", end="  ")
    print()
        
    # optimization loop    
    step = 0
    for n in range(c['epochs']):
        for m in range(c['steps_per_epoch']-1):
            step += 1
            train_step(voxel)
        print(f"Epoch {n}:  ", end="")
        
        for k in losshist.keys():
            losshist[k].append(train_step(voxel, True)[k].numpy())
        for k,v in losshist.items():
            print(f"{k}: {v[-1]:10.1f}", end="  ")
        print()
        
        if n % c['save_modulo'] == 0 and n > 0:
            pickle.dump( {"voxel": voxel.numpy(), "losshist": losshist} , open( os.path.join(conf_global['doc_dir'], f"result_{i}_{n}.pkl" ), "wb" ) )

    print(f"Finished  Loop: {i}")

    # Final documentation
    pickle.dump( {"voxel": voxel.numpy(), "losshist": losshist} , open( os.path.join(conf_global['doc_dir'], f"result_{i}.pkl" ), "wb" ) )
    print("Optimization finished")


# Results
The generated shapes and the initial shapes are stored in the ./results folder.\
The shapes are stored as pickle files as numpy matrices.\
The result.pkl also includes the optimization history.\

The result shape is non binary, meaning material is stored as a float between 0 and 1. For plotting, you have to set a materialno materiak threshold.

We recommend ipyvolume for plotting.